## CT-DRENAR fitting

In [ ]:
import os
from dotenv import load_dotenv
#import nmrglue as ng
import matplotlib.pyplot as plt
import numpy as np
from scipy.constants import pi
from scipy.constants import physical_constants
from scipy.optimize import curve_fit
from utils import gamma
from utils.function import drenar

#### Define data and experiment

In [ ]:
load_dotenv()

# define data location
PREFIX = os.getenv("PREFIX")
LIB = os.getenv("LIB")
EXPNAME = os.getenv("EXPNAME")
EXPNO = os.getenv("EXPNO")
procno = '101'  # @param {type:"string"}

dir_exp = os.path.join(PREFIX, LIB, EXPNAME)
dir_proc = os.path.join(PREFIX, LIB, EXPNAME, EXPNO, 'pdata')
dir_int = os.path.join(PREFIX, LIB, EXPNAME, EXPNO, 'pdata', procno, EXPNAME+'_int.txt')

print("Path to integral list: "+dir_int)
print("Path to experiment data: "+dir_exp)

# define experiment paramters
l0 = 1
spin_rate = 8  # in kHz
t = l0 * 16 / spin_rate  # in ms, 16 rotor cycles per BaBa block

# define constants
mu_0 = physical_constants['vacuum mag. permeability'][0]
hbar = physical_constants['reduced Planck constant'][0]
gamma_I = gamma.P
gamma_S = gamma.P

# define a guess of the distance as initial value for fitting
r_guess = 0.3  # in nm

#### Read integral values and calculate differences

In [ ]:
int_list = []
if True:
    with open(dir_int, 'r') as f:
        for line in f.readlines():
            int_list.append([float(x) for x in line.split()])
    int_list = np.reshape(int_list, (len(int_list[1]), len(int_list)))
    int_list = np.reshape(int_list[0],(2,-1))
else:
    pass # raise error

time_discrete = 16 / spin_rate * l0 * np.linspace(1, int_list.shape[1], num=int_list.shape[1])  # in ms
time_continuous = np.linspace(0, time_discrete[-1], num=100)

print("{:d} steps, longest dephasing time {:.1f} ms.".format(int_list.shape[1], time_discrete[-1]))

print("\nList of integrals: ")
print(int_list)

diff_list = np.zeros(np.shape(int_list)[1])
for i in range(np.shape(int_list)[1]):
    diff_list[i] = 1 - int_list[0][i]/int_list[1][i]

print("\nList of relative differences to reference: ")
print(diff_list)

fig, ax = plt.subplots(figsize=(8,4), constrained_layout=True)
ax.scatter(time_discrete, diff_list)
ax.set_xlabel('Dephasing time (ms)')
ax.set_ylabel('1-S/S0')
plt.show()

#### Fit the DRENAR curve

In [ ]:
points_for_fitting = 3

d_guess = mu_0 / (4*pi) * (gamma_I*gamma_S*hbar) / (2*pi) / (r_guess/10**9)**3 /1000  # in kHz
print('Dipolar coupling constant by guess = {:.3f} kHz'.format(d_guess))

calculator = drenar_parabola()

popt, pconv = curve_fit(calculator, time_discrete[:points_for_fitting], diff_list[:points_for_fitting], p0=d_guess)
#perr = np.sqrt(np.diag(pconv))  # standard deviation
d_opt = popt[0]
print('Effective dipolar coupling constant by fitting = {:.3f} kHz'.format(d_opt))

r_opt = (mu_0 / (4*pi) * (gamma_I*gamma_S*hbar) / (2*pi) / d_opt /1000)**(1/3) * 10**9  # in nm
print('Effective distance r = {:.3f} nm'.format(r_opt))


# plot
diff_opt = calculator(time_continuous, d_opt)
diff_guess = calculator(time_continuous, d_guess)

fig, ax = plt.subplots(figsize=(8,4), constrained_layout=True)
ax.scatter(time_discrete, diff_list)
ax.plot(time_continuous, diff_opt)
#ax.plot(time_axis, redor_guess)
ax.set_xlabel('Dephasing time (ms)')
ax.set_ylabel('1-S/S0')
ax.set_xlim(left=0)
ax.set_ylim(bottom=0, top=max(1,1.1*diff_list.max()))
ax.text(0.5*time_discrete[-1],0.1*diff_list.max(),'Effective dipolar coupling = {:.0f} Hz'.format(1000*d_opt))
plt.show()


Author: Yufei Wu  
Last changed: 2024-11-19  
Email: yufei.wu@itmc.rwth-aachen.de